In [86]:
import requests
import json

# key = 'Px6s6Gi2JgvZRgp9iNRwfV5116cfHSMoDDei8Z0CXsPK8MUBdlaB119wgcx9HDt87Cc%2FGltpy2XnhGnYQ70RiA%3D%3D'
# key2 = 'Px6s6Gi2JgvZRgp9iNRwfV5116cfHSMoDDei8Z0CXsPK8MUBdlaB119wgcx9HDt87Cc/Gltpy2XnhGnYQ70RiA=='

# url = 'http://apis.data.go.kr/1480523/WaterQualityService/getWaterMeasuringList'
# # 사실 10, 11, 12월을 가져와도 되지만 2022 10월 데이터를 이용하여 수질을 측정할것이기 떄문에 제외
# params ={'serviceKey' : f'{key2}', 'pageNo' : '1', 'numOfRows' : '80000', 'resultType' : 'json', 'wmyrList' : '2018,2019,2020,2021,2022', 'wmodList' : '01,02,03,04,05,06,07,08,09' }
# # wmyrList = 연도, wmodList = 월

# response = requests.get(url, params=params)

과거 데이터로 학습을 시켜키는데 이때 범위를 줘서 1(식수원), 0(식수원 안되는것)을 부여 -> 피처와 타겟 데이터 분리, 학습 ->09월달 데이터의 이름을 가져와서 그 값을 대입하면 이것을 식수원으로 사용할 수 있는지 없는지 확인가능

> 1일 트래픽양이 있으니 이걸 csv로 저장해서 사용?

In [87]:
# response

In [88]:
# json 형태로 변환
# result_data = response.json()

In [89]:
# result_data_2 = result_data['getWaterMeasuringList']["item"]

In [90]:
# import csv
# #  newline='' 설정이 없는 경우 row와 row 사이에 뉴라인이 한번 더 들어가게 됨
# with open('water.csv', 'w', encoding='utf-8-sig',newline='')as f:
#     wr = csv.DictWriter(f, fieldnames = result_data_2[0].keys())
#     wr.writeheader()
#     wr.writerows(result_data_2)

In [91]:
import pandas as pd

data = pd.read_csv("water.csv")
data.head(3)

C:\Users\kimsdata\AppData\Local\Temp\ipykernel_26260\3379403615.py:3: DtypeWarning: Columns (23,24,26,27,28,29,30,31,32,33,34,35,36,37,41,42,43,44,45,47,48,49,50,51,52,54,55,56,58,59,60,61,62,64,65,66,67,68,69,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("water.csv")


,ROWNO,PT_NO,PT_NM,ADDR,ORG_NM,WMYR,WMOD,WMWK,LON_DGR,LON_MIN,...,ITEM_CHCL3,ITEM_TOC,ITEM_DEHP,ITEM_ANTIMON,ITEM_DIOX,ITEM_HCHO,ITEM_HCB,ITEM_NI,ITEM_BA,ITEM_SE
0,1,2005A15,이안천,경상북도 상주시 함창읍 금곡리 금곡교,낙동강물환경연구소,2022,3,1회차,128.0,12.0,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2004A90,내성천3-1,경상북도 예천군 용궁면 향석리 성저교,낙동강물환경연구소,2022,3,4회차,128.0,18.0,...,NaN,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1007A41,양화천-1,경기도 여주시 가남면 상활리(상활교),한강유역환경청,2020,3,1회차,127.0,31.0,...,NaN,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 수질에 필요한 데이터 골라오기 

* 환경부에서 제공한 하천의 수질 판단요소와 그 값  
> ITEMPH : ph(수소이온농도)  
ITEMCOD : cod(화학적 산소요구량)  
ITEMTOC : toc(총유기탄소량)  
ITEMSS : ss(부유물질량)  
ITEMDOC : doc(용존유기탄소)  
ITEMTP : tp(총 인)  
ITEMTCOLI : tcoli(총 대장균군)  
ITEMECOLI : ecoli(분원성 대장균군)

</br>
6.5∼8.5	4이하	3이하	25이하	5.0이상	0.04이하	500이하	100이하  

위의 내용으로 타겟데이터를 생성

In [92]:
df_db= data[['PT_NM','WMYR','WMOD','WMWK', 'ITEM_PH','ITEM_BOD','ITEM_COD','ITEM_TOC','ITEM_SS','ITEM_DOC','ITEM_TP','ITEM_TCOLI','ITEM_ECOLI']]
df_db

,PT_NM,WMYR,WMOD,WMWK,ITEM_PH,ITEM_BOD,ITEM_COD,ITEM_TOC,ITEM_SS,ITEM_DOC,ITEM_TP,ITEM_TCOLI,ITEM_ECOLI
0,이안천,2022,3,1회차,7.5,1.8,3.8,2.0,2.7,12.3,0.032,NaN,NaN
1,내성천3-1,2022,3,4회차,7.4,0.5,4.5,2.6,12.4,11.2,0.07,4700.0,53.0
2,양화천-1,2020,3,1회차,7.1,1.5,4.8,2.8,5.4,10.3,0.154,NaN,NaN
3,복하천2-1,2020,3,1회차,7.0,8.0,8.7,3.4,6.6,7.7,0.194,NaN,NaN
4,청미천,2020,3,1회차,7.5,2.3,6.7,5.0,5.4,12.8,0.162,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,합천댐2,2021,7,11,7.4,1.0,3.1,2.1,1.6,7.8,0.01,126.0,5.0
79996,주암댐2,2021,7,11,7.2,0.9,2.9,2.1,2.0,10.1,0.024,485.0,2.0
79997,탑정지2,2021,7,11,8.2,1.4,3.8,2.1,9.8,8.0,0.033,0.0,0.0
79998,보청천4,2021,7,1회차,8.0,0.7,3.5,2.4,11.6,7.8,0.057,58.0,21.0


In [93]:
df_db[df_db['ITEM_PH']>=20]

,PT_NM,WMYR,WMOD,WMWK,ITEM_PH,ITEM_BOD,ITEM_COD,ITEM_TOC,ITEM_SS,ITEM_DOC,ITEM_TP,ITEM_TCOLI,ITEM_ECOLI
16110,천진호,2018,4,1회차,838.0,1.3,6.8,3.7,2.4,8.7,0.016,220.0,0.0
42480,안동3,2018,4,1회차,76.0,1.6,5.0,2.9,5.2,12.9,0.011,35,4
68694,광포호,2018,4,1회차,744.0,2.0,10.4,4.1,10.2,10.4,0.084,25200.0,0.0
73759,금강,2020,4,1회차,26.1,2.2,7.0,4.2,6.0,12.3,0.043,51.0,24.0


In [94]:
df_db = df_db.drop(df_db[df_db['ITEM_BOD'].isnull()].index)
df_db.reset_index(inplace=True, drop=True)
df_db

,PT_NM,WMYR,WMOD,WMWK,ITEM_PH,ITEM_BOD,ITEM_COD,ITEM_TOC,ITEM_SS,ITEM_DOC,ITEM_TP,ITEM_TCOLI,ITEM_ECOLI
0,이안천,2022,3,1회차,7.5,1.8,3.8,2.0,2.7,12.3,0.032,NaN,NaN
1,내성천3-1,2022,3,4회차,7.4,0.5,4.5,2.6,12.4,11.2,0.07,4700.0,53.0
2,양화천-1,2020,3,1회차,7.1,1.5,4.8,2.8,5.4,10.3,0.154,NaN,NaN
3,복하천2-1,2020,3,1회차,7.0,8.0,8.7,3.4,6.6,7.7,0.194,NaN,NaN
4,청미천,2020,3,1회차,7.5,2.3,6.7,5.0,5.4,12.8,0.162,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70507,안계댐,2021,7,15,8.0,2.8,4.0,2.9,3.2,4.4,0.014,208.0,0.0
70508,합천댐2,2021,7,11,7.4,1.0,3.1,2.1,1.6,7.8,0.01,126.0,5.0
70509,주암댐2,2021,7,11,7.2,0.9,2.9,2.1,2.0,10.1,0.024,485.0,2.0
70510,탑정지2,2021,7,11,8.2,1.4,3.8,2.1,9.8,8.0,0.033,0.0,0.0


In [95]:
# df= data[['ITEMPH','ITEMBOD','ITEMCOD','ITEMTOC','ITEMSS','ITEMDOC','ITEMTP','ITEMTCOLI','ITEMECOLI']]

In [96]:
# from sklearn.impute import SimpleImputer
# # 최빈값으로 simpleimputing - > 새로운 값이 들어올 경우에도 필요하므로 모델학습과정에서 해야하지만 결측치가 너무많아서 drink를 계산할 수 없음. 추후 예측하기 위한 데이터도 결측치가 없어야 될 듯
# sim = SimpleImputer()
# data = pd.DataFrame(sim.fit_transform(data), columns=data.columns)
# data

In [97]:
# iterrows와 at을 사용해서 drink에 값 지정
for i, row in df_db.iterrows() :
    if (df_db.at[i, 'ITEM_BOD'] <= 2.0) & (df_db.at[i, 'ITEM_PH'] <= 8.5) & (df_db.at[i, 'ITEM_PH'] >= 6.5):
        df_db.at[i,'drink'] = 1
    else:
        df_db.at[i,'drink'] = 0
#float type으로 나오기 떄문에 형변환
df_db['drink'] = df_db['drink'].astype(int)

df_db['ITEM_ECOLI'] = df_db['ITEM_ECOLI'].str.replace("정량한계미만","0")
df_db['ITEM_SS'] = df_db['ITEM_SS'].str.replace("정량한계미만","0")
df_db['ITEM_TP'] = df_db['ITEM_TP'].str.replace("정량한계미만","0")
df_db['ITEM_TCOLI'] = df_db['ITEM_TCOLI'].str.replace("정량한계미만","0")
df_db = df_db.astype({'ITEM_SS': 'float', 'ITEM_TP': 'float', 'ITEM_TCOLI': 'float', 'ITEM_ECOLI': 'float'})
df_db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70512 entries, 0 to 70511
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PT_NM       70512 non-null  object 
 1   WMYR        70512 non-null  int64  
 2   WMOD        70512 non-null  int64  
 3   WMWK        70512 non-null  object 
 4   ITEM_PH     70511 non-null  float64
 5   ITEM_BOD    70512 non-null  float64
 6   ITEM_COD    70499 non-null  float64
 7   ITEM_TOC    67976 non-null  float64
 8   ITEM_SS     7412 non-null   float64
 9   ITEM_DOC    70512 non-null  float64
 10  ITEM_TP     27506 non-null  float64
 11  ITEM_TCOLI  4215 non-null   float64
 12  ITEM_ECOLI  4029 non-null   float64
 13  drink       70512 non-null  int32  
dtypes: float64(9), int32(1), int64(2), object(2)
memory usage: 7.3+ MB


In [98]:
# df_db.to_csv('C:/Users/kimsdata/Desktop/study/section3/project3/pro3_app/water2.csv', index = None, encoding='utf-8-sig')

In [99]:
df_db['drink'].value_counts()

1    41781
0    28731
Name: drink, dtype: int64

## DB 생성 - DASHBOARD를 위한!!

In [100]:
# import os
# import sqlite3

# DB_FILENAME = 'water.db'
# DB_FILEPATH = os.path.join(os.getcwd(), DB_FILENAME)
# conn = sqlite3.connect(DB_FILENAME)
# cur = conn.cursor()

# cur.execute("DROP TABLE IF EXISTS water")

# cur.execute("""CREATE TABLE water (
#     id INTEGER PRIMARY KEY,
#     area VARCHAR,
#     Year integer,
#     month integer,
#     PH float,
#     COD float,
#     TOC float,
#     SS float,
#     DOC float,
#     TP float,
#     TCOLI float,
#     ECOLI float,
#     drink float);
# """)

# for index, row in df_db.iterrows():
#     cur.execute("""INSERT INTO water (
#        id, area, Year, month, PH, COD, TOC, SS, DOC, TP, TCOLI, ECOLI, drink) values(?,?,?,?,?,?,?,?,?,?,?,?,?)""",
#        (index, row.PT_NM, row.WMYR, row.WMOD, row.ITEM_PH, row.ITEM_COD, row.ITEM_TOC, row.ITEM_SS, row.ITEM_DOC, row.ITEM_TP, row.ITEM_TCOLI, row.ITEM_ECOLI, row.drink))

# conn.commit()
# conn.close()

# MYSQL

In [ ]:
# import os
# import pymysql

# DB_FILENAME = 'water2'
# DB_FILEPATH = os.path.join(os.getcwd(), DB_FILENAME)


# conn = pymysql.connect(
#     user = 'root'
#     passwd = 'rudehd'
#     host = '127.17.0.1'
#     db='water2'
#     charset='utf8'
# )


# conn = sqlite3.connect(DB_FILENAME)
# cur = conn.cursor(pymysql.cursors.DictCursor)

# cur.execute("DROP TABLE IF EXISTS water")

# cur.execute("""CREATE TABLE water (
#     id INTEGER PRIMARY KEY,
#     area VARCHAR,
#     Year integer,
#     month integer,
#     PH float,
#     COD float,
#     TOC float,
#     SS float,
#     DOC float,
#     TP float,
#     TCOLI float,
#     ECOLI float,
#     drink float);
# """)

# for index, row in df_db.iterrows():
#     cur.execute("""INSERT INTO water (
#        id, area, Year, month, PH, COD, TOC, SS, DOC, TP, TCOLI, ECOLI, drink) values(?,?,?,?,?,?,?,?,?,?,?,?,?)""",
#        (index, row.PT_NM, row.WMYR, row.WMOD, row.ITEM_PH, row.ITEM_COD, row.ITEM_TOC, row.ITEM_SS, row.ITEM_DOC, row.ITEM_TP, row.ITEM_TCOLI, row.ITEM_ECOLI, row.drink))

# conn.commit()
# conn.close()

In [101]:
# 사실상 이게 타겟 데이터
df= df_db[['ITEM_PH','ITEM_COD','ITEM_TOC','ITEM_SS','ITEM_DOC','ITEM_TP','ITEM_TCOLI','ITEM_ECOLI', 'drink']]
df

,ITEM_PH,ITEM_COD,ITEM_TOC,ITEM_SS,ITEM_DOC,ITEM_TP,ITEM_TCOLI,ITEM_ECOLI,drink
0,7.5,3.8,2.0,NaN,12.3,NaN,NaN,NaN,1
1,7.4,4.5,2.6,NaN,11.2,NaN,NaN,NaN,1
2,7.1,4.8,2.8,NaN,10.3,NaN,NaN,NaN,1
3,7.0,8.7,3.4,NaN,7.7,NaN,NaN,NaN,0
4,7.5,6.7,5.0,NaN,12.8,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
70507,8.0,4.0,2.9,NaN,4.4,NaN,NaN,NaN,0
70508,7.4,3.1,2.1,NaN,7.8,NaN,NaN,NaN,1
70509,7.2,2.9,2.1,NaN,10.1,NaN,NaN,NaN,1
70510,8.2,3.8,2.1,NaN,8.0,NaN,NaN,NaN,1


In [102]:
# 오브젝트가 있는것을 확인 ordinal 인코딩 필요
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70512 entries, 0 to 70511
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ITEM_PH     70511 non-null  float64
 1   ITEM_COD    70499 non-null  float64
 2   ITEM_TOC    67976 non-null  float64
 3   ITEM_SS     7412 non-null   float64
 4   ITEM_DOC    70512 non-null  float64
 5   ITEM_TP     27506 non-null  float64
 6   ITEM_TCOLI  4215 non-null   float64
 7   ITEM_ECOLI  4029 non-null   float64
 8   drink       70512 non-null  int32  
dtypes: float64(8), int32(1)
memory usage: 4.6 MB


In [103]:
from sklearn.model_selection import train_test_split

target = 'drink'
features = df.drop(columns=[target]).columns
train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=df[target])

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [104]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape


((56409, 8), (56409,), (14103, 8), (14103,))

In [105]:
y_train.value_counts()

1    33424
0    22985
Name: drink, dtype: int64

In [106]:
from sklearn.metrics import accuracy_score, f1_score

base = y_train.mode()[0]
baseline = len(y_train) * [base]
baseline_roc = f1_score(y_train, baseline)
print(f'기준모델의 f1_score {baseline_roc.round(3)} 입니다')

기준모델의 f1_score 0.744 입니다


In [107]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
from category_encoders import OrdinalEncoder
from scipy.stats.distributions import uniform
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, roc_auc_score
import numpy as np

# strategy="most_frequent"

In [108]:
# params = {
#     # "lgbmclassifier__boost_from_average": hp.choice("boost_from_average", ["True","False"]),
#     "lgbmclassifier__max_depth": hp.quniform("max_depth", -1, 12, 1),
#     "lgbmclassifier__min_child_weight": hp.uniform("min_child_weight", 0.01, 10),
#     "lgbmclassifier__n_estimators": hp.quniform("n_estimators", 200, 1000, 10),
#     "lgbmclassifier__colsample_bytree": hp.uniform("colsample_bytree", 0, 1.0),
#     "lgbmclassifier__subsample": hp.uniform("subsample", 0, 1.0),
#     "lgbmclassifier__learning_rate": hp.uniform("learning_rate", 0.01, 0.3),
#     "lgbmclassifier__scale_pos_weight": hp.uniform("scale_pos_weight", 0.5, 5),
#     "lgbmclassifier__reg_alpha": hp.uniform("reg_alpha", 0, 100),
#     "lgbmclassifier__reg_lambda": hp.uniform("reg_lambda", 0, 100),
# }

In [109]:
# def get_pipe(params):
#     params["lgbmclassifier__max_depth"] = int(
#         params["lgbmclassifier__max_depth"]
#     )
#     params["lgbmclassifier__n_estimators"] = int(
#         params["lgbmclassifier__n_estimators"]
#     )  # max_depth는 정수형으로 변환해 줍니다.
#     pipe = make_pipeline(
#         # OrdinalEncoder(),
#         # 평균값으로 대체한다.
#         SimpleImputer(),
#         LGBMClassifier(
#             objective="binary",
#             metric="binary_logloss", # binary_error" 일수도 있음
#             random_state=2,
#             n_jobs=-1,      
#         ),
#     )
#     pipe = pipe.set_params(**params)
#     return pipe

# def fit_and_eval(params):

#     pipe = get_pipe(params)  # 주어진 params로 파이프라인을 만들어 가져옵니다.
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2) # 편향된 데이터의 경우 교차검증 때 불균형인 성능평가가 될수있으므로 섞음
#     # 교차검증 분할기 StratifiedKFold를 이용해 더 세세한값으로 cv를 준다.
#     # 일반적으로 클래스가 있는 분류 문제에서는 층화 K-fold CV를 활용하며 회귀 문제에서는 K-fold CV를 활용
#     score = cross_val_score(pipe, X_train, y_train, cv=skf, scoring="f1")
#     avg_cv_score = np.mean(score)
#     return {"loss": -avg_cv_score, "status": STATUS_OK}

# trials = (Trials()) # Trials() 객체를 fmin에 함께 넣어 실행하면, 실행되는 매 fit의 학습 정보 및 학습 결과가 해당 객체에 모두 저장됩니다.

# best_params = fmin(fn=fit_and_eval, trials=trials, space=params, algo=tpe.suggest, max_evals=30)  # max_evals 횟수만큼 하이퍼파라미터 조합을 탐색합니다.

In [110]:
# print("최적 하이퍼파라미터: ", trials.best_trial["misc"]["vals"])
# print("최적 f1: ", -trials.best_trial["result"]["loss"])

In [111]:
lgbm = make_pipeline(
    # OrdinalEncoder(),
    SimpleImputer(),
    LGBMClassifier(
        boosting_type='gbdt',
        objective="binary",
        metric="f1",
        n_estimators=650,
        random_state=2,
        n_jobs=-1,
        learning_rate=0.10396570732929125,
        colsample_bytree = 0.9222261160173689,
        max_depth=6,
        min_child_weight =9.858478650458594,
        scale_pos_weight =1.0612116731151515,
        boost_from_average=False,
        subsample = 0.33225749039571384,
        reg_alpha = 10.905199654176023,
        reg_lambda = 71.89314201175601,
    ),
)

lgbm.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('lgbmclassifier',
                 LGBMClassifier(boost_from_average=False,
                                colsample_bytree=0.9222261160173689,
                                learning_rate=0.10396570732929125, max_depth=6,
                                metric='f1', min_child_weight=9.858478650458594,
                                n_estimators=650, objective='binary',
                                random_state=2, reg_alpha=10.905199654176023,
                                reg_lambda=71.89314201175601,
                                scale_pos_weight=1.0612116731151515,
                                subsample=0.33225749039571384))])

In [112]:
y_pred_test = lgbm.predict(X_test)
y_pred_roc_test = lgbm.predict_proba(X_test)[:,1]
print(classification_report(y_test, y_pred_test))
print("테스트 accuracy", accuracy_score(y_test, y_pred_test))
print("테스트 f1_score", f1_score(y_test, y_pred_test))
print("테스트 roc_auc score", roc_auc_score(y_test, y_pred_roc_test))

              precision    recall  f1-score   support

           0       0.86      0.79      0.82      5746
           1       0.86      0.91      0.89      8357

    accuracy                           0.86     14103
   macro avg       0.86      0.85      0.85     14103
weighted avg       0.86      0.86      0.86     14103

테스트 accuracy 0.8603134084946465
테스트 f1_score 0.885371814267427
테스트 roc_auc score 0.930751812780697


In [113]:
# import eli5
# from eli5.sklearn import PermutationImportance

# # permuter 정의
# permuter = PermutationImportance(
#     lgbm.named_steps["lgbmclassifier"],  # model 소문자로 써야함
#     scoring="roc_auc",  # metric f1으로하면 드랍컬럼이 많아짐
#     n_iter=5,  # 다른 random seed를 사용하여 10번 반복
#     random_state=42,
# )

# # 스코어를 다시 계산합니다.
# # X_val_enc = lgbm[0].transform(X_test) # 인코딩만 하는것
# # X_val_sim = lgbm[1].transform(X_val_enc) # 인코딩만 하는것
# # SimpleImputer만 이럼
# # X_val_preprocessed = pd.DataFrame(X_val_sim, columns=X_test.columns)

# permuter.fit(X_test, y_test)

In [114]:
# feature_names = X_test.columns.tolist() #  tolist() 함수를 이용해서 리스트로 만들수 있다.
# permutation = pd.Series(permuter.feature_importances_, feature_names).sort_values()

In [115]:
# # 특성별 score을 확인합니다.
# eli5.explain_weights( # show_weight랑 같음
#     permuter,
#     top=None,  # top n 지정 가능, None 일 경우 모든 특성
#     feature_names=feature_names,  # list 형식으로 넣어야 합니다
# )

In [116]:
import pickle

with open('model.pkl','wb') as pickle_file:
    pickle.dump(lgbm, pickle_file)

In [117]:
# with open('model.pkl','rb') as pickle_file:
#     col = pickle.load(pickle_file)

원하는 식수원지 이름 입력하거나 ITEM_PH	/ITEM_COD /ITEM_TOC /ITEM_SS /ITEM_DOC /ITEM_TP /ITEM_TCOLI /ITEM_ECOLI 를 입력

In [118]:
# # 모델 이름 lgbm 값을 넣어줄때 [[]]의 안의 값을 넣어줘야함
# # -------------------각각 입력값을 입력받았을때---------------
# # ls = [[10,10,10,10,10,10,10,10]]
# # result = lgbm.predict(ls)
# # result

# # -------------------이름만 입력받았을때----------------------
# # 분기별로 측정이고 최근이 5월이다.


# area = input()
# re = df_db[(df_db['PT_NM'] == area) & (df_db['WMYR'] == 2022) & (df_db['WMWK'] == "3회차") & (df_db['WMOD'] == 5)]
# # 반드시 순서대로 넣어줄것
# re = re[['ITEM_PH','ITEM_COD','ITEM_TOC','ITEM_SS','ITEM_DOC','ITEM_TP','ITEM_TCOLI','ITEM_ECOLI']]
# result = lgbm.predict(re)
# result

In [123]:
re = df_db[(df_db['PT_NM'] == '금강') & (df_db['WMYR'] == 2022) & (df_db['WMWK'] == "3회차") & (df_db['WMOD'] == 5)]
re = re[['ITEM_PH','ITEM_COD','ITEM_TOC','ITEM_SS','ITEM_DOC','ITEM_TP','ITEM_TCOLI','ITEM_ECOLI']]
result = lgbm.predict(re)
result

array([0])

In [124]:
df_db[(df_db['PT_NM'] == '금강') & (df_db['WMYR'] == 2022) & (df_db['WMWK'] == "3회차") & (df_db['WMOD'] == 5)]

,PT_NM,WMYR,WMOD,WMWK,ITEM_PH,ITEM_BOD,ITEM_COD,ITEM_TOC,ITEM_SS,ITEM_DOC,ITEM_TP,ITEM_TCOLI,ITEM_ECOLI,drink
39977,금강,2022,5,3회차,9.1,5.4,9.8,6.1,NaN,13.7,NaN,360.0,250.0,0
